In [1]:
import psycopg2.pool
from flask import Flask, jsonify


In [2]:


pool = psycopg2.pool.SimpleConnectionPool(1, 20,
                                            host="chimera-db-instance.cfwdn6rdkll5.us-east-1.rds.amazonaws.com",
                                            port="5432",
                                            database="chimera_pay",
                                            user="Chimera_DB",
                                            password="Test12345")



In [3]:
columns = ["phone","Address"]
#

In [3]:
query = "select phone, Address from chimera_user "

In [12]:
query = "insert into chimera_user (phone, Address) values ('+919958498945','ABC Address')"

In [28]:
query = "select phone, Address from chimera_user where phone like '+919958498945'"

In [4]:
conn = pool.getconn()
conn.autocommit = True
with conn.cursor() as cursor:
    cursor.execute(query)
    record = cursor.fetchall()

pool.putconn(conn)
record


[('+919958498955', 'ABC Address'), ('+919958498945', 'ABC Address')]

In [9]:
def execute_enter_data(table_name:str, 
                            column_names:str, 
                            values):
                            
    
    columns_to_enter = ",".join(map(str,column_names))
    values_to_enter  = ",".join(map(lambda x: rf"'{x}'",values))    
    #base query                        
    query = f"insert into {table_name} ({columns_to_enter}) values ({values_to_enter})"


    # return query
    try:
        # Get a connection from the pool
        conn = pool.getconn()
        conn.autocommit = True
        with conn.cursor() as cursor:
            cursor.execute(query)
            return 1
            # record = await cursor.fetchone()
            # return jsonify({"id": record[0], "name": record[1], "cost": record[2]})

    except (Exception, psycopg2.Error) as error :
        print(query)
        print ("Error while inserting data from PostgreSQL -->", error)
        return -1

    finally:
        # Return the connection to the pool
        if(conn):
            pool.putconn(conn)
            print("PostgreSQL connection returned to the pool")

In [10]:
execute_enter_data("chimera_user", ["phone","Address"], ["+919958498965","ABCD"])

PostgreSQL connection returned to the pool


1

In [6]:
def execute_get_data(table_name:str, 
                    column_names:str = None, 
                    where_dict:dict = None):

    if column_names != None:
        columns_to_return = ",".join(map(str,column_names))
    else:
        columns_to_return = "*"
    #base query                        
    query = f"select {columns_to_return} from {table_name}"

    if where_dict!=None:
        where_clause    = " AND ".join(map(lambda x : f"{x} like '{where_dict[x]}'",where_dict))
        query           = query + " where "+where_clause
    # return query

    try:
        # Get a connection from the pool
        conn = pool.getconn()
        conn.autocommit = True
        with conn.cursor() as cursor:
            cursor.execute(query)
            record = cursor.fetchall()
            return record

    except (Exception, psycopg2.Error) as error :
        print(query)
        print ("Error while fetching data from PostgreSQL -->", error)
        return -1

    finally:
        # Return the connection to the pool
        if(conn):
            pool.putconn(conn)
            print("PostgreSQL connection returned to the pool")

In [7]:
execute_get_data("chimera_user", ["phone","Address"], {"phone" : "+919958498965"})

PostgreSQL connection returned to the pool


[('+919958498965', 'ABCD')]

In [8]:
execute_get_data("chimera_user", ["phone","Address"])

PostgreSQL connection returned to the pool


[('+919958498955', 'ABC Address'),
 ('+919958498945', 'ABC Address'),
 ('+919958498965', 'ABCD')]

In [ ]:
async def execute_get_data(table_name, column_names =None, where_dict = None):
    
  try:
    # Get a connection from the pool
    conn = pool.getconn()

    async with conn.cursor() as cursor:
      postgreSQL_select_Query = "select * from postage where id = %s"
      await cursor.execute(postgreSQL_select_Query, (id,))
      record = await cursor.fetchone()
      return jsonify({"id": record[0], "name": record[1], "cost": record[2]})

  except (Exception, psycopg2.Error) as error :
      print ("Error while fetching data from PostgreSQL", error)

  finally:
      # Return the connection to the pool
      if(conn):
          pool.putconn(conn)
          print("PostgreSQL connection returned to the pool")

In [ ]:
async def execute_query(query):
  try:
    # Get a connection from the pool
    conn = pool.getconn()

    async with conn.cursor() as cursor:
      postgreSQL_select_Query = "select * from postage where id = %s"
      await cursor.execute(postgreSQL_select_Query, (id,))
      record = await cursor.fetchone()
      return jsonify({"id": record[0], "name": record[1], "cost": record[2]})

  except (Exception, psycopg2.Error) as error :
      print ("Error while fetching data from PostgreSQL", error)

  finally:
      # Return the connection to the pool
      if(conn):
          pool.putconn(conn)
          print("PostgreSQL connection returned to the pool")